In [86]:
# Replace non ascii in metadata

import json

metadatas = json.load(open('metadata_backup.json'))

ascii_replace_dict = {'â€™': '’', 'â€œ': '“', 'â€': '”', '\u00e2\u20ac\u2122': '’', '\u2019': '\'', '\u201d': '', '\u201c': '', '\u00e2\u20ac' : '-'}
def replace_non_ascii(s):
    for pair_k, pair_v in ascii_replace_dict.items():
        s = s.replace(pair_k, pair_v)
    return s

for metadata in metadatas:
    metadata['file_url'] = replace_non_ascii(metadata['file_url'])
    metadata['title'] = replace_non_ascii(metadata['title'])
    if 'source' in metadata.keys() and metadata['source']:
        metadata['source'] = replace_non_ascii(metadata['source'])

import json
with open('metadata_backup.json', 'w') as f:
    json.dump(metadatas, f)

In [31]:
files = json.load(open('all_files.json'))
all_filenames = [file['filename'].rsplit(".",1)[0] for file in files['data'] if file['filename'].split('.')[-1] == "docx"]

In [104]:
all_filenames = [replace_non_ascii(f) for f in all_filenames]

In [33]:
import json
asst_file = json.load(open('asst_file.json'))

In [105]:
metadatas = json.load(open('metadata_backup.json'))

for metadata in metadatas:
    file_url = metadata['file_url'].rsplit('.', 1)[0]
    title = metadata['title']
    if file_url in all_filenames or title in all_filenames:
        idx = None
        if file_url in all_filenames:
            idx = all_filenames.index(file_url)
        elif title in all_filenames:
            idx = all_filenames.index(title)
        file_OAI_id = files['data'][idx]['id']
        metadata['file_OAI_id'] = file_OAI_id
        metadata['assistant_OAI_id'] = next(pair for pair in asst_file if pair['file_id'] == file_OAI_id)['assistant_id']
    else:
        print(file_url)
        print(title)
        raise RuntimeError('File not found!')

In [106]:
with open('metadata.json', 'w') as f:
    json.dump(metadatas, f)

In [35]:
assistants = [a_f['assistant_id'] for a_f in asst_file]

In [56]:
assistant_summary = json.load(open('assistant_summary.json'))

In [ ]:
# Summary creation

from openai import OpenAI
import time
import re

client = OpenAI()
already_done = [a_s['assistant_id'] for a_s in assistant_summary]

for assistant in assistants:
    if assistant in already_done:
        continue
    print(assistant)
    thread = client.beta.threads.create()

    thread_message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = "Create a very short summary that uses 30 completion_tokens or less… Giving only the information that will be useful when considering whether this file is needed to answer a specific question.",
    )
    run = client.beta.threads.runs.create(
        thread_id = thread.id,
        assistant_id = assistant
    )
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        time.sleep(1)

    messages = client.beta.threads.messages.list(thread_id=thread.id)
    last_msg = max(messages.data, key = lambda x: x.created_at)
    response = re.sub('【.*】', '', last_msg.content[0].text.value)
    assistant_summary.append({'assistant_id': assistant, 'summary': response})
    time.sleep(1)

with open('assistant_summary.json', 'w') as f:
    json.dump(assistant_summary, f)

In [53]:
with open('assistant_summary.json', 'w') as f:
    json.dump(assistant_summary, f)

In [25]:
last_msg.content[0].text.annotations[0].file_citation.quote

'Experiments with Alternate Currents of High Potential and High Frequency\nby Nikola Tesla\n\n\xa0\nThe Inventions Researches and Writings of Nikola Tesla- by Thomas Commerford Martin Editor -\xa0Chapter XXVIIPg 198\n\n\n\n\xa0\n\xa0\nI cannot find words to express how deeply I feel the honor of addressing some of the foremost thinkers of the present time and so many able scientific men engineers and electricians of the country greatest in scientific achievements.\xa0\nThe results which I have the honor to present before such a gathering I cannot call my own. There are among you not a few who can lay better claim than myself on any feature of merit which this work may contain. I need not mention many names which are world-known—names of those among you who are recognized as the leaders in this enchanting science; but one at least I must mention—a name which could not be omitted in a demonstration of this kind. It is a name associated with the most beautiful invention ever made: it is C